# Sequential model using Keras library

Following instructions from here: https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

And using code from here: https://gist.github.com/fchollet/f35fbc80e066a49d65f1688a7e99f069

The eventual fully connected layers will be based on this: https://www.kaggle.com/devm2024/transfer-learning-with-vgg-16-convnet-lb-0-1850/notebook

In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

Using Theano backend.


In [5]:
import os.path

In [11]:
# define variables

# dimensions of our images.
img_width, img_height = 75, 75

top_model_weights_path = '/Users/ilanrotenberg/projects/courses/deeplearning1/nbs/data/statoil/results/bottleneck_fc_model.h5'
train_data_dir = '/Users/ilanrotenberg/projects/courses/deeplearning1/nbs/data/statoil/train/'
validate_data_dir = '/Users/ilanrotenberg/projects/courses/deeplearning1/nbs/data/statoil/validate/'
nb_train_samples = sum(os.path.isfile(os.path.join(train_data_dir + 'iceberg/', f)) for f in os.listdir(train_data_dir + 'iceberg/'))
nb_train_samples += sum(os.path.isfile(os.path.join(train_data_dir + 'ship/', f)) for f in os.listdir(train_data_dir + 'ship/'))
nb_validate_samples = sum(os.path.isfile(os.path.join(validate_data_dir + 'iceberg/', f)) for f in os.listdir(validate_data_dir + 'iceberg/'))
nb_validate_samples += sum(os.path.isfile(os.path.join(validate_data_dir + 'ship/', f)) for f in os.listdir(validate_data_dir + 'ship/'))
epochs = 50
batch_size = 16



In [21]:
nb_train_samples

1122

In [25]:
def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)
    
    #Build the Vgg16 network
    model = applications.VGG16(include_top=False, weights='imagenet')
    
    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size = (img_width, img_height),
        batch_size = batch_size,
        class_mode = None,
        shuffle = False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'w'), 
            bottleneck_features_train)
    
    generator = datagen.flow_from_directory(
        validate_data_dir,
        target_size = (img_width, img_height),
        batch_size = batch_size,
        class_mode = None,
        shuffle = False)
    bottleneck_features_validate = model.predict_generator(
        generator, nb_validate_samples // batch_size)
    np.save(open('bottleneck_features_validate.npy', 'w'), 
            bottleneck_features_validate)
        
        
def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy'))
    train_labels = np.array(
        [0] * (len(train_data) / 2) + [1] * (len(train_data) / 2))
        
    validation_data = np.load(open('bottleneck_features_validate.npy'))
    validation_labels = np.array(
        [0] * (len(validation_data) / 2) + [1] * (len(validation_data) / 2))
    
        
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
        
    model.compile(optimizer='rmsprop', loss = 'binary_crossentropy', metrics=['accuracy'])
    
    model.fit(train_data, train_labels,
             epochs = epochs,
             batch_size = batch_size,
             validation_data = (validation_data, validation_labels))
    model.save_weights(top_model_weights_path)

In [16]:
save_bottleneck_features()

58597376/58889256 [============================>.] - ETA: 0sFound 1122 images belonging to 2 classes.
Found 482 images belonging to 2 classes.


In [27]:
train_data = np.load(open('bottleneck_features_train.npy'))
train_data.shape

(1120, 2, 2, 512)

In [26]:
train_top_model()

Train on 1120 samples, validate on 480 samples
Epoch 1/50
1120/1120 [==============================] - 0s - loss: 0.6249 - acc: 0.6875 - val_loss: 0.5689 - val_acc: 0.6979
Epoch 2/50
1120/1120 [==============================] - 0s - loss: 0.5173 - acc: 0.7420 - val_loss: 0.5250 - val_acc: 0.7292
Epoch 3/50
1120/1120 [==============================] - 0s - loss: 0.5001 - acc: 0.7384 - val_loss: 0.4792 - val_acc: 0.7625
Epoch 4/50
1120/1120 [==============================] - 0s - loss: 0.4881 - acc: 0.7562 - val_loss: 0.4912 - val_acc: 0.7667
Epoch 5/50
1120/1120 [==============================] - 0s - loss: 0.4657 - acc: 0.7687 - val_loss: 0.5256 - val_acc: 0.7417
Epoch 6/50
1120/1120 [==============================] - 0s - loss: 0.4581 - acc: 0.7670 - val_loss: 0.4811 - val_acc: 0.7688
Epoch 7/50
1120/1120 [==============================] - 0s - loss: 0.4548 - acc: 0.7857 - val_loss: 0.4935 - val_acc: 0.7646
Epoch 8/50
1120/1120 [==============================] - 0s - loss: 0.4448 - ac

In [28]:
import pandas as pd


In [32]:
test_path = train_data_dir + '../test.json'


[Errno 2] No such file or directory: '/Users/ilanrotenberg/projects/courses/deeplearning1/nbs/data/statoil/train/../test.json - test.json'
/Users/ilanrotenberg/projects/courses/deeplearning1/nbs/statoil-nb


In [34]:
test_data = pd.read_json(test_path)

In [35]:
test_data.shape

(8424, 4)

In [36]:
test_data[:5]

,band_1,band_2,id,inc_angle
0,"[-15.863251, -15.201077, -17.887735, -19.17248...","[-21.629612, -21.142353, -23.908337, -28.34524...",5941774d,34.966400
1,"[-26.0589694977, -26.0589694977, -26.058969497...","[-25.7542076111, -25.7542076111, -25.754207611...",4023181e,32.615072
2,"[-14.1410999298, -15.0642414093, -17.375520706...","[-14.745639801, -14.5904102325, -14.3626976013...",b20200e4,37.505433
3,"[-12.167478, -13.706167, -16.54837, -13.572674...","[-24.32222, -26.375538, -24.096739, -23.8769, ...",e7f018bb,34.473900
4,"[-23.3745937347, -26.0271816254, -28.121963501...","[-25.7223434448, -27.0115776062, -23.149162292...",4371c8c3,43.918874


In [37]:
X_band_1 = np.array([np.array(band).astype('float32').reshape(img_width,img_height) for band in test_data.band_1])

In [38]:
X_band_2 = np.array([np.array(band).astype('float32').reshape(img_width,img_height) for band in test_data.band_2])
X_band_3 = (X_band_1 + X_band_2) / 2
X_band = np.concatenate([X_band_1[:,:,:,np.newaxis],
                        X_band_2[:,:,:,np.newaxis],
                        X_band_3[:,:,:,np.newaxis]],axis=-1)

In [39]:
preds = model.predict(X_band)

NameError: name 'model' is not defined

# Create a new model

create a new model that does not rely on Imagenet database (Since data is not similar to imagenet)

#### Steps:
1. import data
2. separate training and validation sets
3. get data and labels for both sets
4. create simple model to start understanding data
5. TBD

In [48]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Lambda
from keras import applications
import os.path
import pandas as pd
import random
from keras.utils import to_categorical

In [3]:
# define variables

# dimensions of our images.
img_width, img_height = 75, 75

top_model_weights_path = '/Users/ilanrotenberg/projects/courses/deeplearning1/nbs/data/statoil/results/bottleneck_fc_model.h5'
train_data_dir = '/Users/ilanrotenberg/projects/courses/deeplearning1/nbs/data/statoil/train/'
validate_data_dir = '/Users/ilanrotenberg/projects/courses/deeplearning1/nbs/data/statoil/validate/'
nb_train_samples = sum(os.path.isfile(os.path.join(train_data_dir + 'iceberg/', f)) for f in os.listdir(train_data_dir + 'iceberg/'))
nb_train_samples += sum(os.path.isfile(os.path.join(train_data_dir + 'ship/', f)) for f in os.listdir(train_data_dir + 'ship/'))
nb_validate_samples = sum(os.path.isfile(os.path.join(validate_data_dir + 'iceberg/', f)) for f in os.listdir(validate_data_dir + 'iceberg/'))
nb_validate_samples += sum(os.path.isfile(os.path.join(validate_data_dir + 'ship/', f)) for f in os.listdir(validate_data_dir + 'ship/'))
epochs = 50
batch_size = 16

In [4]:
train_path = train_data_dir + '../train.json'
test_path = train_data_dir + '../test.json'
train_batch = pd.read_json(train_path)
test_batch = pd.read_json(test_path)

In [5]:
train_batch[:5]

,band_1,band_2,id,inc_angle,is_iceberg
0,"[-27.878361, -27.15416, -28.668615, -29.537971...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0
1,"[-12.242375, -14.920305, -14.920363, -12.66633...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0
2,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",58b2aaa0,45.2859,1
3,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",4cfc3a18,43.8306,0
4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",271f93f4,35.6256,0


In [6]:
validate_batch = train_batch.sample(frac=0.3)

In [7]:
validate_batch[:5]

,band_1,band_2,id,inc_angle,is_iceberg
747,"[-25.409029, -22.566938, -19.967796, -25.40916...","[-31.714437, -31.714485, -28.791969, -28.79201...",efc00a93,42.4438,0
993,"[-27.015923, -26.10083, -23.38764, -21.157799,...","[-26.700037, -28.039034, -25.013737, -26.70015...",8b2054f8,38.0144,0
521,"[-24.811668, -25.078993, -18.923769, -20.86201...","[-28.333494, -31.099594, -27.223236, -25.93412...",07ecb0dd,43.2611,1
484,"[-16.920742, -16.920742, -18.400295, -15.74773...","[-23.706635, -23.706635, -28.453804, -25.19920...",35a8236c,41.1346,0
912,"[-12.625193, -12.275377, -12.049828, -12.92800...","[-20.64369, -20.64369, -20.349295, -21.42687, ...",000d71b1,34.8374,0


In [18]:
new_train_batch = train_batch[~train_batch.index.isin(validate_batch.index)]
new_train_batch[:5]

,band_1,band_2,id,inc_angle,is_iceberg
0,"[-27.878361, -27.15416, -28.668615, -29.537971...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0
1,"[-12.242375, -14.920305, -14.920363, -12.66633...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0
3,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",4cfc3a18,43.8306,0
4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",271f93f4,35.6256,0
5,"[-20.769371, -20.769434, -25.906025, -25.90602...","[-29.288746, -29.712593, -28.884804, -28.88480...",b51d18b5,36.9034,1


In [21]:
validate_batch.to_json(path_or_buf = train_data_dir+'../validate.json')
new_train_batch.to_json(path_or_buf = train_data_dir+'../train_new.json')

In [14]:
train_path = train_data_dir+'../train_new.json'
validate_path = train_data_dir+'../validate.json'

In [19]:
new_train_batch.inc_angle = new_train_batch.inc_angle.apply(lambda x:np.nan if x =='na' else x)

In [15]:
train_data = pd.read_json(train_path)

In [16]:
train_data[:5]

,band_1,band_2,id,inc_angle,is_iceberg
0,"[-27.878361, -27.15416, -28.668615, -29.537971...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0
1,"[-12.242375, -14.920305, -14.920363, -12.66633...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0
10,"[-21.397552, -19.753859, -23.426783, -24.65221...","[-26.72291, -27.418192, -27.787899, -25.774536...",3aac67cd,44.624,1
100,"[-20.04884, -19.469616, -20.510244, -19.61095,...","[-29.742329, -26.374287, -25.490265, -25.49031...",66348d03,41.1342,0
1001,"[-22.34741, -22.156555, -25.308764, -24.530453...","[-24.782082, -24.047678, -24.782185, -27.45301...",3062fca8,39.9627,1


In [20]:
new_train_batch[:5]

,band_1,band_2,id,inc_angle,is_iceberg
0,"[-27.878361, -27.15416, -28.668615, -29.537971...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0
1,"[-12.242375, -14.920305, -14.920363, -12.66633...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0
3,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",4cfc3a18,43.8306,0
4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",271f93f4,35.6256,0
5,"[-20.769371, -20.769434, -25.906025, -25.90602...","[-29.288746, -29.712593, -28.884804, -28.88480...",b51d18b5,36.9034,1


In [22]:
train_band_1 = np.array([np.array(band).astype('float32').reshape(img_width,img_height) for band in new_train_batch.band_1])
train_band_2 = np.array([np.array(band).astype('float32').reshape(img_width,img_height) for band in new_train_batch.band_2])

In [25]:
validate_band_1 = np.array([np.array(band).astype('float32').reshape(img_width,img_height) for band in validate_batch.band_1])
validate_band_2 = np.array([np.array(band).astype('float32').reshape(img_width,img_height) for band in validate_batch.band_2])

test_band_1 = np.array([np.array(band).astype('float32').reshape(img_width,img_height) for band in test_batch.band_1])
test_band_2 = np.array([np.array(band).astype('float32').reshape(img_width,img_height) for band in test_batch.band_2])

train_band_3 = 

In [26]:
X_train = np.concatenate([train_band_1[:,:,:,np.newaxis],
                         train_band_2[:,:,:,np.newaxis]],axis=-1)
X_validate = np.concatenate([validate_band_1[:,:,:,np.newaxis],
                            validate_band_2[:,:,:,np.newaxis]], axis=-1)
X_test = np.concatenate([test_band_1[:,:,:,np.newaxis],
                        test_band_2[:,:,:,np.newaxis]], axis=-1)

In [27]:
y_train = np.array([np.array(iceberg).astype('uint8') for iceberg in new_train_batch.is_iceberg])

In [28]:
y_train[:5]

array([0, 0, 0, 0, 1], dtype=uint8)

In [29]:
y_validate = np.array([np.array(iceberg).astype('uint8') for iceberg in validate_batch.is_iceberg])

In [31]:
X_train.shape

(1123, 75, 75, 2)

In [32]:
X_validate.shape

(481, 75, 75, 2)

In [34]:
def onehot(x):
    return to_categorical(x)


In [35]:
y_train = onehot(y_train)

In [36]:
y_train[:5]

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.]])

In [37]:
y_validate = onehot(y_validate)

In [40]:
np.moveaxis(X_train,3,1).shape

(1123, 2, 75, 75)

In [41]:
X_train = np.moveaxis(X_train,3,1)
X_validate = np.moveaxis(X_validate,3,1)
X_test = np.moveaxis(X_test,3,1)

In [42]:
X_train.shape

(1123, 2, 75, 75)

In [43]:
mean_trainx = X_train.mean().astype(np.float32)
std_trainx = X_train.std().astype(np.float32)

In [44]:
def norm_input(x): return ((x-mean_trainx)/std_trainx)

In [66]:
def get_linear_model():
    model = Sequential([
        Lambda(norm_input, input_shape = (2,img_height,img_width)),
        Flatten(),
        Dense(2, activation='softmax')
    ])
    model.compile(optimizer = 'Adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [67]:
lm = get_linear_model()

/Users/ilanrotenberg/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:629: UserWarning: `output_shape` argument not specified for layer lambda_5 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 2, 75, 75)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [68]:
gen = ImageDataGenerator()
batches = gen.flow(X_train, y_train, batch_size=32)
test_batches = gen.flow(X_validate, y_validate, batch_size=32)

In [69]:
lm.fit_generator(batches, batches.n, epochs = 1, 
                validation_data = test_batches, validation_steps = test_batches.n)

Epoch 1/1
1123/1123 [==============================] - 5s - loss: 0.2031 - acc: 0.9405 - val_loss: 0.6113 - val_acc: 0.7381


In [65]:
y_validate[:5]

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.]])

In [70]:
lm.optimizer.lr=0.1

In [71]:
lm.fit_generator(batches, batches.n, epochs = 1, 
                validation_data = test_batches, validation_steps = test_batches.n)

Epoch 1/1
1123/1123 [==============================] - 5s - loss: 0.0097 - acc: 1.0000 - val_loss: 0.6896 - val_acc: 0.7325


In [72]:
lm.optimizer.lr=0.01

In [73]:
lm.fit_generator(batches, batches.n, epochs = 4, 
                validation_data = test_batches, validation_steps = test_batches.n)

Epoch 1/4
1123/1123 [==============================] - 5s - loss: 0.0036 - acc: 1.0000 - val_loss: 0.7550 - val_acc: 0.7319
Epoch 2/4
1123/1123 [==============================] - 5s - loss: 0.0017 - acc: 1.0000 - val_loss: 0.8067 - val_acc: 0.7236
Epoch 3/4
1123/1123 [==============================] - 5s - loss: 8.7078e-04 - acc: 1.0000 - val_loss: 0.8597 - val_acc: 0.7276
Epoch 4/4
1123/1123 [==============================] - 5s - loss: 4.6663e-04 - acc: 1.0000 - val_loss: 0.9088 - val_acc: 0.7280


In [75]:
def get_fc_model():
    model = Sequential([
        Lambda(norm_input, input_shape = (2,img_height,img_width)),
        Flatten(),
        Dense(512, activation ='softmax'),
        Dense(2, activation='softmax')
    ])
    model.compile(optimizer = 'Adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [76]:
fc = get_fc_model()

/Users/ilanrotenberg/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:629: UserWarning: `output_shape` argument not specified for layer lambda_6 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 2, 75, 75)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [77]:
fc.fit_generator(batches, batches.n, epochs = 1, 
                validation_data = test_batches, validation_steps = test_batches.n)

Epoch 1/1
1123/1123 [==============================] - 310s - loss: 0.6647 - acc: 0.6153 - val_loss: 0.6577 - val_acc: 0.5987


In [78]:
fc.optimizer.lr = 0.1

In [79]:
fc.fit_generator(batches, batches.n, epochs = 1, 
                validation_data = test_batches, validation_steps = test_batches.n)

Epoch 1/1
1123/1123 [==============================] - 219s - loss: 0.5991 - acc: 0.6720 - val_loss: 0.5729 - val_acc: 0.7028


In [80]:
def get_cnn_model():
    model = Sequential([
        Lambda(norm_input, input_shape = (2,img_height,img_width)),
        Convolution2D(32,3,3,activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(32,3,3,activation='relu'),
        MaxPooling(),
        Convolution2D(64,3,3,activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3,activation='relu'),
        MaxPooling(),
        Convolution2D(128,3,3,activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(128,3,3,activation='relu'),
        MaxPooling(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation ='relu'),
        BatchNormalization(),
        Dense(2, activation='softmax')
    ])
    model.compile(optimizer = 'Adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [81]:
cm = get_cnn_model